# Hamlet Analysis
## Hadley Rose

Using Pyspark to work with Hamlet text: Import/Clean/Transform Data, EDA, NLP, and Sentiment Analysis

* [x] [Import Data](#Import-Data)  
* [ ] [Clean/Transform Data](#Clean/Transform-Data)  
* [ ] EDA  
* [ ] NLP/Sentiment Analysis  

Resources:

* [Pyspark Udemy course](https://www.udemy.com/course/spark-and-python-for-big-data-with-pyspark/)
* [Project Gutenberg Hamlet text](https://www.gutenberg.org/files/1524/1524-0.txt)  
* [Folger's Online Shakespeare Hamlet with line numbers](https://shakespeare.folger.edu/shakespeares-works/hamlet/)  
* [Wolfram Hamlet Data](https://datarepository.wolframcloud.com/resources/Hamlet)  
* [SO: SparkFiles for data from HTTP](https://stackoverflow.com/a/57019702)  
* [SO: id column](https://stackoverflow.com/questions/32086578/how-to-add-row-id-in-pyspark-dataframes/35948427#35948427)  
* [SO: slicing a pyspark df](https://stackoverflow.com/a/52819758)  
* [SO: pyspark column with conditional values](https://stackoverflow.com/a/51565873)  
* [SO: create column using regular expression matching](https://stackoverflow.com/a/46547701)  
* [SO: replace column value conditionally](https://stackoverflow.com/a/44773899)  
* [Mode Analytics: Window Functions](https://mode.com/sql-tutorial/sql-window-functions/)  
* [SO: Pyspark window functions](https://stackoverflow.com/a/52835995)  
* [Apache Drill: SQL Window Functions](https://drill.apache.org/docs/sql-window-functions-introduction/)  
* [SO: Tracking previous row's value, consecutive zeros](https://stackoverflow.com/a/55003211)  
* [SO: Comparing to SQL null is undefined](https://stackoverflow.com/a/39121638)  
* [Blog article about SQL NULL](https://www.xaprb.com/blog/2006/05/18/why-null-never-compares-false-to-anything-in-sql/)  
* [SO: Why != not working in pyspark](https://stackoverflow.com/a/54154697)

### Import Data

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
# get Hamlet txt from Project Gutenberg
url = 'https://www.gutenberg.org/files/1524/1524-0.txt'
spark.sparkContext.addFile(url)
df = spark.read.text("file://"+SparkFiles.get('1524-0.txt'))
df.show()

+--------------------+
|               value|
+--------------------+
|                    |
|Project Gutenberg...|
|                    |
|This eBook is for...|
|most other parts ...|
|restrictions what...|
|under the terms o...|
|eBook or online a...|
|United States, yo...|
|are located befor...|
|                    |
|                    |
|                    |
|       Title: Hamlet|
|                    |
|Author: William S...|
|                    |
|Release Date: Nov...|
|Last Updated: Sep...|
|                    |
+--------------------+
only showing top 20 rows



In [4]:
# return one line
df.collect()[3][0]

'This eBook is for the use of anyone anywhere in the United States and'

### Clean/Transform Data

In [5]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, regexp_extract, lag, lead

In [6]:
# create id column to "splice" dataframe as if in pandas
df = df.withColumn('id', monotonically_increasing_id())
df = df.withColumnRenamed('value', 'line')
df.show()

+--------------------+---+
|                line| id|
+--------------------+---+
|                    |  0|
|Project Gutenberg...|  1|
|                    |  2|
|This eBook is for...|  3|
|most other parts ...|  4|
|restrictions what...|  5|
|under the terms o...|  6|
|eBook or online a...|  7|
|United States, yo...|  8|
|are located befor...|  9|
|                    | 10|
|                    | 11|
|                    | 12|
|       Title: Hamlet| 13|
|                    | 14|
|Author: William S...| 15|
|                    | 16|
|Release Date: Nov...| 17|
|Last Updated: Sep...| 18|
|                    | 19|
+--------------------+---+
only showing top 20 rows



In [7]:
# actual play contents start at id = 31
df.where(col('id') > 30).show()

+--------------------+---+
|                line| id|
+--------------------+---+
|THE TRAGEDY OF HA...| 31|
|                    | 32|
|                    | 33|
|                    | 34|
|                    | 35|
|by William Shakes...| 36|
|                    | 37|
|                    | 38|
|                    | 39|
|                    | 40|
|                    | 41|
|                    | 42|
|            Contents| 43|
|                    | 44|
|               ACT I| 45|
|Scene I. Elsinore...| 46|
|Scene II. Elsinor...| 47|
|Scene III. A room...| 48|
|Scene IV. The pla...| 49|
|Scene V. A more r...| 50|
+--------------------+---+
only showing top 20 rows



In [8]:
# actual play starts at id = 108
df = df.where(col('id') >= 108)
df.show()

+--------------------+---+
|                line| id|
+--------------------+---+
|               ACT I|108|
|                    |109|
|SCENE I. Elsinore...|110|
|                    |111|
| Enter Francisco ...|112|
|                    |113|
|BARNARDO. Who’s t...|114|
|                    |115|
|FRANCISCO. Nay, a...|116|
|                    |117|
|BARNARDO. Long li...|118|
|                    |119|
|FRANCISCO. Barnardo?|120|
|                    |121|
|       BARNARDO. He.|122|
|                    |123|
|FRANCISCO. You co...|124|
|                    |125|
|BARNARDO. ’Tis no...|126|
|                    |127|
+--------------------+---+
only showing top 20 rows



In [9]:
# create line number column
# line whenever a character starts to speak, e.g., BARNARDO.
# line numbers restart after SCENE indicator, e.g., SCENE I.
# should we write to mongodb where we do {act1: scene1: context: [context stuff, applies to: lines], line_no: line}?
# create context indicator?

In [10]:
# remove blank lines
df = df.filter(df['line'] != '')
df.show()

+--------------------+---+
|                line| id|
+--------------------+---+
|               ACT I|108|
|SCENE I. Elsinore...|110|
| Enter Francisco ...|112|
|BARNARDO. Who’s t...|114|
|FRANCISCO. Nay, a...|116|
|BARNARDO. Long li...|118|
|FRANCISCO. Barnardo?|120|
|       BARNARDO. He.|122|
|FRANCISCO. You co...|124|
|BARNARDO. ’Tis no...|126|
|FRANCISCO. For th...|128|
|           at heart.|129|
|BARNARDO. Have yo...|131|
|FRANCISCO. Not a ...|133|
|BARNARDO. Well, g...|135|
|rivals of my watc...|136|
| Enter Horatio an...|138|
|FRANCISCO. I thin...|140|
|HORATIO. Friends ...|142|
|MARCELLUS. And li...|144|
+--------------------+---+
only showing top 20 rows



In [11]:
# create indicator column
# rows like ACT I have indicator ACT I, use regex group to extract 'ACT Roman Numeral'
# rows like SCENE I have indicator SCENE I, use regex group to extract 'SCENE Roman Numeral'
# rows starting with CHARACTER. are lines where that character is speaking, indicate as character name
df.withColumn('indicator', 
              when(col('line').contains('ACT'), regexp_extract(col('line'), '(ACT\s[I]+)', 1))
              .when(col('line').contains('SCENE'), regexp_extract(col('line'), '(SCENE\s[I]+)', 1))
              .when(col('line').rlike('^([A-Z]+)\.\s\w+'), regexp_extract(col('line'), '^([A-Z]+)\.\s\w+', 1))
              .otherwise(0)).show()

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|               ACT I|108|    ACT I|
|SCENE I. Elsinore...|110|  SCENE I|
| Enter Francisco ...|112|        0|
|BARNARDO. Who’s t...|114| BARNARDO|
|FRANCISCO. Nay, a...|116|FRANCISCO|
|BARNARDO. Long li...|118| BARNARDO|
|FRANCISCO. Barnardo?|120|FRANCISCO|
|       BARNARDO. He.|122| BARNARDO|
|FRANCISCO. You co...|124|FRANCISCO|
|BARNARDO. ’Tis no...|126|        0|
|FRANCISCO. For th...|128|FRANCISCO|
|           at heart.|129|        0|
|BARNARDO. Have yo...|131| BARNARDO|
|FRANCISCO. Not a ...|133|FRANCISCO|
|BARNARDO. Well, g...|135| BARNARDO|
|rivals of my watc...|136|        0|
| Enter Horatio an...|138|        0|
|FRANCISCO. I thin...|140|FRANCISCO|
|HORATIO. Friends ...|142|  HORATIO|
|MARCELLUS. And li...|144|MARCELLUS|
+--------------------+---+---------+
only showing top 20 rows



In [12]:
# regex to identify lines most likely to be stage directions
# based on looking at the top 20 rows of the df, stage directions start with a single space followed by a word
# starting with a capital letter
df.where(col('line').rlike('^\s[A-Z]')).show()

+--------------------+----+
|                line|  id|
+--------------------+----+
| Enter Francisco ...| 112|
| Enter Horatio an...| 138|
|        Enter Ghost.| 185|
|     Re-enter Ghost.| 273|
| Enter Claudius K...| 332|
| Laertes, Voltema...| 333|
| Enter Horatio, M...| 467|
| Enter Laertes an...| 622|
|     Enter Polonius.| 668|
| Enter Hamlet, Ho...| 763|
|        Enter Ghost.| 808|
| Enter Ghost and ...| 886|
| Enter Horatio an...|1002|
| Enter Polonius a...|1116|
|      Enter Ophelia.|1206|
| Enter King, Quee...|1257|
|     Enter Polonius.|1297|
| Enter Polonius w...|1327|
| Enter Hamlet, re...|1433|
| Enter Rosencrant...|1510|
+--------------------+----+
only showing top 20 rows



In [13]:
# create indicator column
# rows like ACT I have indicator ACT I, use regex group to extract 'ACT Roman Numeral'
# rows like SCENE I have indicator SCENE I, use regex group to extract 'SCENE Roman Numeral'
# rows starting with CHARACTER. are lines where that character is speaking, indicate as character name
# replaced \w with . in regex pattern to find lines similar to BARNARDO. 'Tis ...
# rows most likely to be stage direction as DIRECTION
df = df.withColumn('indicator', 
                   when(col('line').contains('ACT'), regexp_extract(col('line'), '(ACT\s[I]+)', 1))
                   .when(col('line').contains('SCENE'), regexp_extract(col('line'), '(SCENE\s[I]+)', 1))
                   .when(col('line').rlike('^([A-Z]+)\.\s.+'), regexp_extract(col('line'), '^([A-Z]+)\.\s.+', 1))
                   .when(col('line').rlike('^\s[A-Z]'), 'DIRECTION')
                   .otherwise(0))
df.show()

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|               ACT I|108|    ACT I|
|SCENE I. Elsinore...|110|  SCENE I|
| Enter Francisco ...|112|DIRECTION|
|BARNARDO. Who’s t...|114| BARNARDO|
|FRANCISCO. Nay, a...|116|FRANCISCO|
|BARNARDO. Long li...|118| BARNARDO|
|FRANCISCO. Barnardo?|120|FRANCISCO|
|       BARNARDO. He.|122| BARNARDO|
|FRANCISCO. You co...|124|FRANCISCO|
|BARNARDO. ’Tis no...|126| BARNARDO|
|FRANCISCO. For th...|128|FRANCISCO|
|           at heart.|129|        0|
|BARNARDO. Have yo...|131| BARNARDO|
|FRANCISCO. Not a ...|133|FRANCISCO|
|BARNARDO. Well, g...|135| BARNARDO|
|rivals of my watc...|136|        0|
| Enter Horatio an...|138|DIRECTION|
|FRANCISCO. I thin...|140|FRANCISCO|
|HORATIO. Friends ...|142|  HORATIO|
|MARCELLUS. And li...|144|MARCELLUS|
+--------------------+---+---------+
only showing top 20 rows



In [14]:
# attempting to indicate which character the wrapped line belongs to
df.where(col('indicator') == 0).show()

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|           at heart.|129|        0|
|rivals of my watc...|136|        0|
|           [_Exit._]|152|        0|
|take hold of him ...|167|        0|
|Therefore I have ...|168|        0|
|this night, That ...|169|        0|
|    and speak to it.|170|        0|
|are so fortified ...|175|        0|
|the pole, Had mad...|180|        0|
|burns, Marcellus ...|181|        0|
|BARNARDO It would...|195|        0|
|that fair and war...|200|        0|
|sometimes march? ...|201|        0|
|     [_Exit Ghost._]|209|        0|
|something more th...|214|        0|
|and true avouch O...|217|        0|
|When he th’ambiti...|222|        0|
|angry parle He sm...|223|        0|
|stalk hath he gon...|226|        0|
|gross and scope o...|229|        0|
+--------------------+---+---------+
only showing top 20 rows



In [15]:
# quick inspection of top 20 rows where indicator = 0 shows that some stage directions have gotten through
# we can replace indicator value with DIRECTION for lines containing [_Exit
df = df.withColumn('indicator', 
                   when(col('line').rlike('\[_Exit'), 'DIRECTION')
                   .otherwise(df['indicator']))
df.show(40)

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|               ACT I|108|    ACT I|
|SCENE I. Elsinore...|110|  SCENE I|
| Enter Francisco ...|112|DIRECTION|
|BARNARDO. Who’s t...|114| BARNARDO|
|FRANCISCO. Nay, a...|116|FRANCISCO|
|BARNARDO. Long li...|118| BARNARDO|
|FRANCISCO. Barnardo?|120|FRANCISCO|
|       BARNARDO. He.|122| BARNARDO|
|FRANCISCO. You co...|124|FRANCISCO|
|BARNARDO. ’Tis no...|126| BARNARDO|
|FRANCISCO. For th...|128|FRANCISCO|
|           at heart.|129|        0|
|BARNARDO. Have yo...|131| BARNARDO|
|FRANCISCO. Not a ...|133|FRANCISCO|
|BARNARDO. Well, g...|135| BARNARDO|
|rivals of my watc...|136|        0|
| Enter Horatio an...|138|DIRECTION|
|FRANCISCO. I thin...|140|FRANCISCO|
|HORATIO. Friends ...|142|  HORATIO|
|MARCELLUS. And li...|144|MARCELLUS|
|FRANCISCO. Give y...|146|FRANCISCO|
|MARCELLUS. O, far...|148|MARCELLUS|
|FRANCISCO. Barnar...|150|FRANCISCO|
|           [_Exit._]|152|DIRECTION|
|

In [16]:
df.where(col('indicator') == 0).show()

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|           at heart.|129|        0|
|rivals of my watc...|136|        0|
|take hold of him ...|167|        0|
|Therefore I have ...|168|        0|
|this night, That ...|169|        0|
|    and speak to it.|170|        0|
|are so fortified ...|175|        0|
|the pole, Had mad...|180|        0|
|burns, Marcellus ...|181|        0|
|BARNARDO It would...|195|        0|
|that fair and war...|200|        0|
|sometimes march? ...|201|        0|
|something more th...|214|        0|
|and true avouch O...|217|        0|
|When he th’ambiti...|222|        0|
|angry parle He sm...|223|        0|
|stalk hath he gon...|226|        0|
|gross and scope o...|229|        0|
|              state.|230|        0|
|same strict and m...|233|        0|
+--------------------+---+---------+
only showing top 20 rows



Inspection of the top 20 rows where `indicator = 0` shows that there are cases where the lines indicating a character is speaking do not consistently have a period after the character's name.

For example, `id = 195` has a line beginning with 'BARNARDO It would...'.

Based on what we've seen of the dataset, we can identify all lines starting with multiple capital letters that do not equal the words ACT or SCENE as a line indicating a character is speaking.

We can apply the regexp_extract logic above to replace the indicator with the character name instead of 0.

In [17]:
# lines beginning with all caps like BARNARNDO but not necessarily being followed by period should have
# indicator value of the all capital text, i.e. the character name
# DONE: make sure ACT and SCENE aren't affected by this code
# had to add second [A-Z] in rlike to ensure rows were only selected if they had multiple capital letters in the
# first word, not just one capital letter to begin a new sentence
df = df.withColumn('indicator', 
                   when((col('line').rlike('^([A-Z][A-Z]+)')) & (col('line').rlike('(ACT)|(SCENE)') == False), 
                        regexp_extract(col('line'), '^([A-Z]+)', 1))
                   .otherwise(df['indicator']))
df.where(col('id') == 195).show()
# shows that ACT # and SCENE # indicators are not affected
df.show()

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|BARNARDO It would...|195| BARNARDO|
+--------------------+---+---------+

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|               ACT I|108|    ACT I|
|SCENE I. Elsinore...|110|  SCENE I|
| Enter Francisco ...|112|DIRECTION|
|BARNARDO. Who’s t...|114| BARNARDO|
|FRANCISCO. Nay, a...|116|FRANCISCO|
|BARNARDO. Long li...|118| BARNARDO|
|FRANCISCO. Barnardo?|120|FRANCISCO|
|       BARNARDO. He.|122| BARNARDO|
|FRANCISCO. You co...|124|FRANCISCO|
|BARNARDO. ’Tis no...|126| BARNARDO|
|FRANCISCO. For th...|128|FRANCISCO|
|           at heart.|129|        0|
|BARNARDO. Have yo...|131| BARNARDO|
|FRANCISCO. Not a ...|133|FRANCISCO|
|BARNARDO. Well, g...|135| BARNARDO|
|rivals of my watc...|136|        0|
| Enter Horatio an...|138|DIRECTION|
|FRANCISCO. I thin...|140|FRANCISCO|
|HORATIO. Friends ...|142|  HORATIO|


In [18]:
df.where(col('indicator') == 0).show()

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|           at heart.|129|        0|
|rivals of my watc...|136|        0|
|take hold of him ...|167|        0|
|Therefore I have ...|168|        0|
|this night, That ...|169|        0|
|    and speak to it.|170|        0|
|are so fortified ...|175|        0|
|the pole, Had mad...|180|        0|
|burns, Marcellus ...|181|        0|
|that fair and war...|200|        0|
|sometimes march? ...|201|        0|
|something more th...|214|        0|
|and true avouch O...|217|        0|
|When he th’ambiti...|222|        0|
|angry parle He sm...|223|        0|
|stalk hath he gon...|226|        0|
|gross and scope o...|229|        0|
|              state.|230|        0|
|same strict and m...|233|        0|
|the land, And why...|234|        0|
+--------------------+---+---------+
only showing top 20 rows



Inspection of top 20 rows where indicator = 0 now seems to be wrapped lines. With this **assumption**, we will use a window function to replace their indicator values with the character name the lines should be associated with.

Pseudo-code to express logic of what we want the window function to do.

```python
# pseudo-code
for row in df:
    if row['indicator'] == 0:
        if previous_row['indicator'] != 0:
            row['indicator'] = lag('indicator', 1)
```

In [19]:
from pyspark.sql import Window

In [20]:
# use window function to replace wrapped lines with indicator = CHARACTER instead of 0
# comparison needed to be != '0' instead of != 0
# NOTE: comparison to SQL null is not possible, so need to include condition for when lag value is not null
win = Window.orderBy('id')

condition = ((lag(col('indicator'), 1).over(win).isNotNull()) & (lag(col('indicator'), 1).over(win) != '0') & 
             (col('indicator') == 0))

df.withColumn('test', 
                   when(condition, lag(df['indicator'], 1).over(win))
                   ).show(40)

+--------------------+---+---------+---------+
|                line| id|indicator|     test|
+--------------------+---+---------+---------+
|               ACT I|108|    ACT I|     null|
|SCENE I. Elsinore...|110|  SCENE I|     null|
| Enter Francisco ...|112|DIRECTION|     null|
|BARNARDO. Who’s t...|114| BARNARDO|     null|
|FRANCISCO. Nay, a...|116|FRANCISCO|     null|
|BARNARDO. Long li...|118| BARNARDO|     null|
|FRANCISCO. Barnardo?|120|FRANCISCO|     null|
|       BARNARDO. He.|122| BARNARDO|     null|
|FRANCISCO. You co...|124|FRANCISCO|     null|
|BARNARDO. ’Tis no...|126| BARNARDO|     null|
|FRANCISCO. For th...|128|FRANCISCO|     null|
|           at heart.|129|        0|FRANCISCO|
|BARNARDO. Have yo...|131| BARNARDO|     null|
|FRANCISCO. Not a ...|133|FRANCISCO|     null|
|BARNARDO. Well, g...|135| BARNARDO|     null|
|rivals of my watc...|136|        0| BARNARDO|
| Enter Horatio an...|138|DIRECTION|     null|
|FRANCISCO. I thin...|140|FRANCISCO|     null|
|HORATIO. Fri

The window function as I have defined it above only replaces the first 0 indicator. This is fine for lines where there is only 1 additional wrapped lines, but multiple consecutive wrapped lines retain 0 indicator after first wrapped line is replaced. I found a StackOverflow post where the code was intending to do something similar to what I am. This post helped a lot, and I was able to achieve my goal using the code below.

In [21]:
from pyspark.sql.functions import last, lit

In [22]:
# this SO post helped immensely, utilized the comment on answer to make sure this code was as succinct as possible
# https://stackoverflow.com/a/55003211
# modify indicator column where if indicator == 0, then replace with null, else keep indicator value
# then for same indicator column, if indicator is null, replace with last non-null value
# id lines of 167-170 should be MARCELLUS instead of 0
win = Window.orderBy('id')
df = df.withColumn('indicator', 
                   when(col('indicator') == 0, lit(None))
                   .otherwise(df['indicator'])).withColumn('indicator',
                                                           last('indicator', ignorenulls=True).over(win))
df.show(40)

+--------------------+---+---------+
|                line| id|indicator|
+--------------------+---+---------+
|               ACT I|108|    ACT I|
|SCENE I. Elsinore...|110|  SCENE I|
| Enter Francisco ...|112|DIRECTION|
|BARNARDO. Who’s t...|114| BARNARDO|
|FRANCISCO. Nay, a...|116|FRANCISCO|
|BARNARDO. Long li...|118| BARNARDO|
|FRANCISCO. Barnardo?|120|FRANCISCO|
|       BARNARDO. He.|122| BARNARDO|
|FRANCISCO. You co...|124|FRANCISCO|
|BARNARDO. ’Tis no...|126| BARNARDO|
|FRANCISCO. For th...|128|FRANCISCO|
|           at heart.|129|FRANCISCO|
|BARNARDO. Have yo...|131| BARNARDO|
|FRANCISCO. Not a ...|133|FRANCISCO|
|BARNARDO. Well, g...|135| BARNARDO|
|rivals of my watc...|136| BARNARDO|
| Enter Horatio an...|138|DIRECTION|
|FRANCISCO. I thin...|140|FRANCISCO|
|HORATIO. Friends ...|142|  HORATIO|
|MARCELLUS. And li...|144|MARCELLUS|
|FRANCISCO. Give y...|146|FRANCISCO|
|MARCELLUS. O, far...|148|MARCELLUS|
|FRANCISCO. Barnar...|150|FRANCISCO|
|           [_Exit._]|152|DIRECTION|
|

In [23]:
# TODO: define line numbers
# (LOGIC: new line whenever indicator switches from character to character or when character is speaking &
# line wraps & line starts with capital letter indicating a new sentence in character's statement)
# I don't really want repeating groups, but for analysis with pyspark, should I have act and scene indicators for
# all lines following declaration of act and scene?
# Still considering key-value pairs instead, but need to think through approach